
Learning outcomes
------------------------------

- Construct a steady-state flowsheet using the IDAES unit model library
- Connecting unit models in a  flowsheet using Arcs
- Using the SequentialDecomposition tool to initialize a flowsheet with recycle
- Fomulate and solve an optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 


Problem Statement
------

Hydrodealkylation is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups,. In this
example, toluene will be reacted with hydrogen gas at high temperatures
 to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**


This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will neglect for this example.

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.

The flowsheet that we will be using for this module is shown below with the stream conditions. We will be processing 872 TPY of toluene and 19 TPY of hydrogen to produce at least 369 TPY of benzene. As shown in the flowsheet, there are two flash tanks, F101 to separate out the non-condensibles and F102 to further separate the benzene-toluene mixture to improve the benzene purity.  Note that typically a distillation column is required to obtain high purity benzene but that is beyond the scope of this workshop. The non-condensibles separated out in F101 will be partially recycled back to M101 and the rest will be either purged or combusted for power generation.We will assume ideal gas for this flowsheet. The properties required for this module is available in the same directory:

- hda_ideal_VLE.py
- hda_reaction.py

The state variables chosen for the property package are **flows of component by phase, temperature and pressure**. The components considered are: **toluene, hydrogen, benzene and methane**. Therefore, every stream has 8 flow variables, 1 temperature and 1 pressure variable. 

![](module_2_flowsheet.png)




Importing required pyomo and idaes components
-----------
To construct a flowsheet, we will need several components from the pyomo and idaes package. Let us first import the components we need from Pyomo.

In [ ]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           minimize,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition

From idaes, we will be needing the FlowsheetBlock and the following unit models:
- Mixer
- <span style="color:blue">**Heater**</span>
- StoichiometricReactor
- <span style="color:blue">**Flash**</span>
- Separator (splitter) 
- PressureChanger

In [ ]:
from idaes.core import FlowsheetBlock

In [ ]:
from idaes.unit_models import (PressureChanger,
                               StoichiometricReactor,
                               Mixer,
                               Separator as Splitter)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, import the remaining unit models highlighted in blue above and run the cell using `Shift+Enter` after typing in the code. 
</div>


In [ ]:
from idaes.unit_models import Heater, Flash

We will also be needing some utility tools to put together the flowsheet and calculate the degrees of freedom. 

In [ ]:
from idaes.unit_models.pressure_changer import ThermodynamicAssumption
from idaes.ui.report import degrees_of_freedom

Importing required thermo and reaction package
-----------


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
The final set of imports are to import the thermo and reaction package for the HDA process. We have created a custom thermo package that assumes Ideal Gas with support for VLE. The reaction package here is very simple as we will be using only a StochiometricReactor and the reaction package consists the stochiometric coefficients for the reaction. Import the following modules and they are in the same directory as this jupyter notebook:
      <ul>
         <li>hda_ideal_VLE as thermo_props</li>
         <li>hda_reaction as reaction_props </li>
      </ul>
</div>

In [ ]:
import hda_ideal_VLE as thermo_props
import hda_reaction as reaction_props

Constructing the Flowsheet
----------------------------------

We have now imported all the components, unit models, and property modules we need to construct a flowsheet. Let us first create a ConcreteModel object as we did in Module 1. 

In [ ]:
m = ConcreteModel()

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, add a FlowsheetBlock object to the concrete model. Please look at Module 1 on how to create a FlowsheetBlock and set dynamic to `False`. Once you have typed in the code, please use Shift+Enter to run the cell. 
</div>


In [ ]:
m.fs = FlowsheetBlock(default={"dynamic": False})

We need to add the thermo properties and reaction properties for this flowsheet object. 

In [ ]:
m.fs.thermo_params = thermo_props.HDAParameterBlock()
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
        default={"property_package": m.fs.thermo_params})

Adding Unit Models
-----

Let us start adding the unit models we have imported to the flowsheet. Here, we are adding the Mixer (assigned a name M101) and a Heater (assigned a name H101). Note that, all unit models need to be given a property package argument. In addition to that, there are several arguments depending on the unit model, please refer to the documentation for more details. For example, the Mixer unit model here is given a `list` consisting of names to the three inlets. 

In [ ]:
m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["toluene_feed", "hydrogen_feed", "vapor_recycle"]})

m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": True})

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us now add the StoichiometricReactor(assign the name R101) and pass the following arguments:
      <ul>
         <li>"property_package": m.fs.thermo_params</li>
         <li>"reaction_package": m.fs.reaction_params </li>
         <li>"has_heat_of_reaction": True </li>
         <li>"has_heat_transfer": True</li>
         <li>"has_pressure_change": False</li>
      </ul>
</div>

In [ ]:
m.fs.R101 = StoichiometricReactor(
            default={"property_package": m.fs.thermo_params,
                     "reaction_package": m.fs.reaction_params,
                     "has_heat_of_reaction": True,
                     "has_heat_transfer": True,
                     "has_pressure_change": False})

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us now add the Flash(assign the name F101) and pass the following arguments:
      <ul>
         <li>"property_package": m.fs.thermo_params</li>
         <li>"has_heat_transfer": True</li>
         <li>"has_pressure_change": False</li>
      </ul>
</div>

In [ ]:
m.fs.F101 = Flash(default={"property_package": m.fs.thermo_params,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})

Let us now add the Splitter(S101), PressureChanger(C101) and the second Flash(F102). 

In [ ]:
m.fs.S101 = Splitter(default={"property_package": m.fs.thermo_params,
                               "ideal_separation": False,
                               "outlet_list": ["purge", "recycle"]})
    

m.fs.C101 = PressureChanger(default={
            "property_package": m.fs.thermo_params,
            "compressor": True,
            "thermodynamic_assumption": ThermodynamicAssumption.isothermal})
    
m.fs.F102 = Flash(default={"property_package": m.fs.thermo_params,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

Connecting Unit Models using Arcs
-----
We have now added all the unit models we need to the flowsheet. However, we have not yet specifed how the units are to be connected. To do this, we will be using the `Arc` which is a pyomo component that takes in two arguments: `source` and `destination`. Let us connect the outlet of the mixer(M101) to the inlet of the heater(H101). 

In [ ]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Add connections to the rest of the flowsheet. Note that the Flash unit model has two outlets called, `liq_outlet` and `vap_outlet`. We will need to add the following connections:
      <ul>
         <li>H101 outlet to R101 inlet</li>
         <li>R101 outlet to F101 inlet</li>
         <li>F101 vap_outlet to S101 inlet</li>
         <li>S101 recycle to C101 inlet</li>
         <li>C101 outlet to M101 vapor_recycle</li>
         <li>F101 liq_outlet to F102 inlet</li>
      </ul>
</div>

![](module_2_flowsheet.png) 


In [ ]:
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet,
               destination=m.fs.M101.vapor_recycle)
m.fs.s10 = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.F102.inlet)

We have now connected the unit model block using the arcs. However, each of these arcs link to ports on the two unit models that are connected. In this case, the ports consist of the state variables that need to be linked between the unit models. Pyomo provides a convenient method to write these equality constraints for us between two ports and this is done as follows:

In [ ]:
TransformationFactory("network.expand_arcs").apply_to(m)

Adding expressions to compute purity and operating costs
---
In this section, we will add a few Expressions that allows us to evaluate the performance. For this flowsheet, we are interested in the purity of the product Benzene stream (i.e. the mole fraction) and the operating cost which is a sum of the cooling and heating cost. 

Let us first add an Expression to compute the mole fraction of benzene in the `vap_outlet` of F102 which is our product stream. Please note that the var flow_mol_phase_comp has the index - [time, phase, component]. As this is a steady-state flowsheet, the time index by default is 0. The valid phases are ["Liq", "Vap"]. Similarly the valid component list is ["benzene", "toluene", "hydrogen", "methane"].

In [ ]:
m.fs.purity = Expression(
        expr=m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] /
        (m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]
         + m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))

Now, let us add an expression to compute the cooling cost assuming a cost of 0.212E-4 $/kW. Note that cooling utility is required for the reactor (R101) and the first flash (F101). 

In [ ]:
m.fs.cooling_cost = Expression(expr=0.212e-7 * -m.fs.F101.heat_duty[0] +
                                   0.212e-7 * -m.fs.R101.heat_duty[0])

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Add an expression to compute the heating cost. Assign the utility cost as follows:
      <ul>
         <li>2.2E-4 dollars/kW for H101</li>
         <li>1.9E-4 dollars/kW for F102</li>
      </ul>
Note that the heat duty is in units of watt (J/s). 
</div>


In [ ]:
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0] +
                                   1.9e-7 * m.fs.F102.heat_duty[0])

Let us now add an expression to compute the total operating cost per year. 

In [ ]:
m.fs.operating_cost = Expression(expr=(3600 * 24 * 365 *
                                           (m.fs.heating_cost +
                                            m.fs.cooling_cost)))

Fixing feed conditions
---

Let us first check how many degrees of freedom exist for this flowsheet using the `degrees_of_freedom` tool we imported earlier. 

In [ ]:
print(degrees_of_freedom(m))

We will now be fixing the toluene feed stream to the conditions shown in the flowsheet above. Please note that though this is a pure toluene feed, the remaining components are still assigned a very small value to help with convergence and initializing. 

In [ ]:
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(0.30)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.temperature.fix(303.2)
m.fs.M101.toluene_feed.pressure.fix(350000)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Fix the hydrogen feed to the following conditions using the syntax above as a guide in the next cell:
      <ul>
         <li>F<sub>H2</sub> = 0.30 mol/s</li>
         <li>F<sub>CH4</sub> = 0.02 mol/s</li>
         <li>Remaining components = 1e-5 mol/s</li>
         <li>T = 303.2 K</li>
         <li>P = 350000 Pa</li>
      </ul>
Use Shift+Enter to run the cell once you have typed in your code. 
</div>


In [ ]:
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(0.30)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(0.02)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.hydrogen_feed.temperature.fix(303.2)
m.fs.M101.hydrogen_feed.pressure.fix(350000)

Fixing unit model specifications
---

We will now be fixing the operating conditions for the unit models in the flowsheet. 

In [ ]:
m.fs.H101.outlet.temperature.fix(600)

For the StoichiometricReactor, we have to define the conversion in terms of toluene. This requires us to create a new variable for specifying the conversion and adding a Constraint that defines the conversion with respect to toluene. The second degree of freedom for the reactor is to define the heat duty. In this case, let us assume the reactor to be adiabatic i.e. Q = 0. 

In [ ]:
m.fs.R101.conversion = Var(initialize=0.75, bounds=(0, 1))

m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion*m.fs.R101.inlet.
    flow_mol_phase_comp[0, "Vap", "toluene"] ==
    (m.fs.R101.inlet.flow_mol_phase_comp[0, "Vap", "toluene"] -
     m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))

m.fs.R101.conversion.fix(0.75)
m.fs.R101.heat_duty.fix(0)

The Flash conditions for F101 can be set as follows. 

In [ ]:
m.fs.F101.vap_outlet.temperature.fix(300.0)
m.fs.F101.deltaP.fix(0)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Set the conditions for Flash F102 to the following conditions:
      <ul>
         <li>T = 375 K</li>
         <li>deltaP = -200000</li>
      </ul>

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [ ]:
m.fs.F102.vap_outlet.temperature.fix(375)
m.fs.F102.deltaP.fix(-200000)

Let us fix the purge split fraction to 20% and the outlet pressure of the compressor is set to 350000 Pa. 

In [ ]:
m.fs.S101.split_fraction[0, "purge"].fix(0.2)
m.fs.C101.outlet.pressure.fix(350000)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now defined all the feed conditions and the inputs required for the unit models. The system should now have 0 degrees of freedom i.e. should be a square problem. Please check that the degrees of freedom is 0. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [ ]:
print(degrees_of_freedom(m))

Scaling factors for this problem. 

In [ ]:
m.fs.H101.control_volume.scaling_factor_energy = 1e-3
m.fs.R101.control_volume.scaling_factor_energy = 1e-3
m.fs.F101.control_volume.scaling_factor_energy = 1e-3
m.fs.C101.control_volume.scaling_factor_energy = 1e-3
m.fs.F102.control_volume.scaling_factor_energy = 1e-3

Initialization
------------------

This section will demonstrate how to use the built-in sequential decomposition tool to initialize our flowsheet. Let us first create an object for the SequentialDecomposition. 

In [ ]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 5

Show how to run first steps

In [ ]:
G = seq.create_graph(m)
heu_result = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

Display tear set and order

In [ ]:
for o in heu_result:
    print(o.name)

In [ ]:
for o in order:
    print(o[0].name)

The SequentialDecomposition tool has determined that the tear stream is the mixer outlet. We will need to provide a reasonable guess for this. 

![](module_2_tear_stream.png) 


In [ ]:
tear_guesses = {
        "flow_mol_phase_comp": {
                (0, "Vap", "benzene"): 1e-5,
                (0, "Vap", "toluene"): 1e-5,
                (0, "Vap", "hydrogen"): 0.30,
                (0, "Vap", "methane"): 0.02,
                (0, "Liq", "benzene"): 1e-5,
                (0, "Liq", "toluene"): 0.30,
                (0, "Liq", "hydrogen"): 1e-5,
                (0, "Liq", "methane"): 1e-5},
        "temperature": {0: 303},
        "pressure": {0: 350000}}

We now need to pass the guesses for the tear stream to the SD tool. 

In [ ]:
seq.set_guesses_for(m.fs.H101.inlet, tear_guesses)

Next, we need to tell the tool what to do to initialise each unit

In [ ]:
def function(unit):
        unit.initialize(outlvl=1)

Run the SequentialDecomposition tool

In [ ]:
seq.run(m, function)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now initialized the flowsheet. Let us run the flowsheet in a simulation mode to look at the results. To do this, we will need to create a solver object and please follow the same instructions as from Module 1. Solve the model. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>



In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}
solver.options = {'tol': 1e-6, 'max_iter': 5000}
results = solver.solve(m, tee=True)

Analyze the results
-------------------------

Let us look at the total operating cost at first. 

In [ ]:
m.fs.operating_cost.display()

For this operating cost, what is the production rate and purity we are able to achieve? 

In [ ]:
m.fs.F102.vap_outlet.display()
m.fs.purity.display()

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We are able to see that in F102, the specifications that we set do not yield the desired results. Let us look at the composition of the liq_outlet from F102. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>


In [ ]:
m.fs.F102.liq_outlet.display()

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
How much benzene are we loosing in the F101 vapor outlet stream? 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>


In [ ]:
m.fs.F101.vap_outlet.display()

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
You can querry additional variables here if you like. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>


Optimization
--------------------------

To formulate an optimization problem, we must first define an objective function we are trying to minimize, total annual operating cost in this instance. 

In [ ]:
m.fs.objective = Objective(sense=minimize, expr=m.fs.operating_cost)

For this problem, our decision variables are as follows:
- H101 outlet temperature
- R101 cooling duty provided
- <span style="color:blue">**F101 outlet temperature**</span>
- <span style="color:blue">**F102 outlet temperature**</span>
- <span style="color:blue">**F102 deltaP or operating pressure of the flash tank**</span>


We will need to unfix these variables and this can be done as follows:

In [ ]:
m.fs.H101.outlet.temperature.unfix()
m.fs.R101.heat_duty.unfix()

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us now unfix the remaining variables that are highlighted in blue above. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>



In [ ]:
m.fs.F101.vap_outlet.temperature.unfix()
m.fs.F102.vap_outlet.temperature.unfix()
m.fs.F102.deltaP.unfix()

Next, we need to set bounds on these variables. Let us first set the variable bound for the H101 outlet temperature as shown below:

In [ ]:
m.fs.H101.outlet.temperature[0].setlb(500)
m.fs.H101.outlet.temperature[0].setub(600)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us set the variable bounds for the remaining variables as follows:
      <ul>
         <li>R101 outlet temperature [600, 800] K</li>
         <li>F101 outlet temperature [300, 450] K</li>
         <li>F102 outlet temperature [273, 450] K</li>
         <li>F102 outlet pressure [105000, 110000] Pa</li>
      </ul>

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [ ]:
m.fs.R101.outlet.temperature[0].setlb(600)
m.fs.R101.outlet.temperature[0].setub(800)
m.fs.F101.vap_outlet.temperature[0].setlb(300.0)
m.fs.F101.vap_outlet.temperature[0].setub(450.0)
m.fs.F102.vap_outlet.temperature[0].setlb(273.0)
m.fs.F102.vap_outlet.temperature[0].setub(450.0)
m.fs.F102.vap_outlet.pressure[0].setlb(105000)
m.fs.F102.vap_outlet.pressure[0].setub(110000)

We also need to add additional performance constraints such that we minimize the overhead loss of benzene in F101 and meet a minimum flow rate at a given purity in the product stream from F102. Let us first add the constraint to limit the overhead loss of benzene in the vapor stream of F101 to less than or equal to 5% of the amount of benzene in the reactor outlet. 

In [ ]:
m.fs.overhead_loss = Constraint(expr=m.fs.S101.purge.flow_mol_phase_comp[0, "Vap", "benzene"] <=
        0.05*m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "benzene"])

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, let us add the constraint such that we are producing at least 0.15 mol/s of benzene in the product stream which is the vapor outlet of F102. Let us name this constraint as m.fs.product_flow. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [ ]:
m.fs.product_flow = Constraint(
        expr=m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] >=
        0.15)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, let us add the final constraint on product purity or the mole fraction of benzene in the product stream such that it is at least greater than 80%. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [ ]:
m.fs.product_purity = Constraint(expr=m.fs.purity >= 0.80)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now defined the optimization problem. We will need to create a solver object if we have not created it earlier. You can now solve the model. Please see earlier modules on how to create a solver object. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>



In [ ]:
results = solver.solve(m, tee=True)

Optimization Results
---
Display the results and product specifications

In [ ]:
m.fs.operating_cost.display()
m.fs.F102.vap_outlet.display()
m.fs.F102.liq_outlet.display()
m.fs.S101.purge.display()

Display optimal values for the decision variables

In [ ]:
m.fs.H101.outlet.temperature.display()
m.fs.R101.heat_duty.display()
m.fs.F101.vap_outlet.temperature.display()
m.fs.F102.vap_outlet.temperature.display()
m.fs.F102.vap_outlet.pressure.display()